In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

ENEDIS_PATH = "/content/drive/MyDrive/Hackathon_2025/datasets/clean/enedis_clean_final.csv"
ADEME_PATH  = "/content/drive/MyDrive/Hackathon_2025/datasets/clean/ademe_final.csv"
OUTPUT_PATH = "/content/drive/MyDrive/Hackathon_2025/datasets/clean/dbz.csv"

# ------------ LOAD ------------
df_enedis = pd.read_csv(ENEDIS_PATH)
df_ademe  = pd.read_csv(ADEME_PATH)

# Harmoniser le nom de colonne
df_enedis.rename(columns={"adress_ban": "address_ban"}, inplace=True)
df_ademe.rename(columns={"adresse_ban": "address_ban"}, inplace=True)

df_enedis["address_ban"] = df_enedis["address_ban"].astype(str)
df_ademe["address_ban"]  = df_ademe["address_ban"].astype(str)

# Extraire l'année ADEME
df_ademe["annee_ademe"] = pd.to_datetime(
    df_ademe["date_derniere_modification_dpe"], errors="coerce"
).dt.year

# ------------ GROUPBY ANNEES ------------
enedis_group = df_enedis.groupby("address_ban")["annee"].unique().to_dict()
ademe_group  = df_ademe.groupby("address_ban")["annee_ademe"].unique().to_dict()

# ------------ MATCHING PAR PROXIMITÉ ------------
rows = []

for address, years_enedis in enedis_group.items():

    years_enedis = sorted([y for y in years_enedis if pd.notna(y)])
    years_ademe  = sorted([y for y in ademe_group.get(address, []) if pd.notna(y)])

    if len(years_enedis) == 0 or len(years_ademe) == 0:
        continue

    years_ademe_available = years_ademe.copy()

    for y_e in years_enedis:

        if len(years_ademe_available) == 0:
            break

        diffs = [abs(y_e - y_a) for y_a in years_ademe_available]
        best_idx = int(np.argmin(diffs))
        y_a_best = years_ademe_available[best_idx]

        # ==== EXTRACTION DES LIGNES COMPLETES ====
        enedis_rows = df_enedis[(df_enedis["address_ban"] == address) &
                                (df_enedis["annee"] == y_e)]

        ademe_rows  = df_ademe[(df_ademe["address_ban"] == address) &
                               (df_ademe["annee_ademe"] == y_a_best)]

        # Si plusieurs lignes dans une base → on garde tout
        for _, row_e in enedis_rows.iterrows():
            for _, row_a in ademe_rows.iterrows():

                merged_row = {}

                # Colonnes ENEDIS préfixées
                for col in df_enedis.columns:
                    merged_row[f"enedis_{col}"] = row_e[col]

                # Colonnes ADEME préfixées
                for col in df_ademe.columns:
                    merged_row[f"ademe_{col}"] = row_a[col]

                # Ajout des années matchées
                merged_row["annee_enedis"] = y_e
                merged_row["annee_ademe_matched"] = y_a_best

                rows.append(merged_row)

        years_ademe_available.pop(best_idx)

# ------------ DATAFRAME FINAL ------------
df_matched = pd.DataFrame(rows)
df_matched.to_csv(OUTPUT_PATH, index=False)

print("✔ Matching COMPLET terminé")
print("📁 Résultat sauvegardé dans :", OUTPUT_PATH)
print(df_matched.head(5))
print(f"\nNombre total de paires (ENEDIS ↔ ADEME) : {len(df_matched):,}")


✔ Matching COMPLET terminé
📁 Résultat sauvegardé dans : /content/drive/MyDrive/Hackathon_2025/datasets/clean/dbz.csv
                                  enedis_address_ban  enedis_ban_lat  \
0        1 All G Anthonioz de Gaulle 93260 Les Lilas       48.879755   
1  1 All Jean Baptiste Camille Corot 76120 Le Gra...       49.402653   
2             1 Allee Ambroise Pare 26200 Montélimar       44.559523   
3         1 Allee Andre Ampere 93110 Rosny-sous-Bois       48.882279   
4              1 Allee Andre Crozillac 33610 Canéjan       44.750613   

   enedis_ban_lon  enedis_ban_score     enedis_ban_id  \
0        2.413890          0.783820  93045_4700_00001   
1        1.037925          0.804955  76322_0728_00001   
2        4.729666          0.955455  26198_0123_00001   
3        2.487403          0.957888  93064_0060_00001   
4       -0.643899          0.957355  33090_3300_00001   

                                enedis_adresse_clean  enedis_annee  \
0            1 ALLEE G ANTHONIOZ DE G

In [ ]:
print(df_matched.columns)

Index(['enedis_address_ban', 'enedis_ban_lat', 'enedis_ban_lon',
       'enedis_ban_score', 'enedis_ban_id', 'enedis_adresse_clean',
       'enedis_annee', 'enedis_numero_de_voie', 'enedis_type_de_voie',
       'enedis_libelle_de_voie', 'enedis_code_commune', 'enedis_nom_commune',
       'enedis_nombre_de_logements',
       'enedis_consommation_annuelle_totale_de_l_adresse_mwh',
       'enedis_adresse', 'enedis_code_departement', 'enedis_code_region',
       'enedis_conso_par_logement_MWh', 'enedis_conso_par_logement_kWh',
       'enedis_numero_de_voie_num', 'ademe_numero_dpe', 'ademe_address_ban',
       'ademe_adresse_brut', 'ademe_numero_voie_ban', 'ademe_nom_rue_ban',
       'ademe_nom_commune_ban', 'ademe_code_postal_ban',
       'ademe_code_insee_ban', 'ademe_surface_habitable_logement',
       'ademe_etiquette_dpe', 'ademe_etiquette_ges',
       'ademe_conso_5_usages_par_m2_ep', 'ademe_conso_5_usages_ep',
       'ademe_conso_5_usages_par_m2_ef', 'ademe_conso_5_usages_ef',
      

On dégage "enedis_adresse_clean" "enedis_ban_score" "enedis_ban_type" "enedis_source" "enedis_adresse"
"ademe_address_ban" "ademe_adresse_brut"
"ademe_adresse_brut.1" "ademe_code_postal_brut

ademe_code_postal_brut', 'ademe_nom_commune_brut',
       'ademe_date_derniere_modification_dpe.1', 'ademe_adresse_ban_clean',
       enedis_annee

In [ ]:
cols_to_drop = [
    "enedis_adresse_clean", "enedis_ban_score", "enedis_ban_type",
    "enedis_source", "enedis_adresse",
    "ademe_address_ban", "ademe_adresse_brut",
    "ademe_adresse_brut.1", "ademe_code_postal_brut",
    "ademe_nom_commune_brut",
    "ademe_date_derniere_modification_dpe.1",
    "ademe_adresse_ban_clean",
    "enedis_annee"
]

df_matched = df_matched.drop(columns=[c for c in cols_to_drop if c in df_matched.columns])

# ------------ SAVE FINAL ------------
df_matched.to_csv(OUTPUT_PATH, index=False)

print(df_matched.columns)

Index(['enedis_address_ban', 'enedis_ban_lat', 'enedis_ban_lon',
       'enedis_ban_id', 'enedis_numero_de_voie', 'enedis_type_de_voie',
       'enedis_libelle_de_voie', 'enedis_code_commune', 'enedis_nom_commune',
       'enedis_nombre_de_logements',
       'enedis_consommation_annuelle_totale_de_l_adresse_mwh',
       'enedis_code_departement', 'enedis_code_region',
       'enedis_conso_par_logement_MWh', 'enedis_conso_par_logement_kWh',
       'enedis_numero_de_voie_num', 'ademe_numero_dpe',
       'ademe_numero_voie_ban', 'ademe_nom_rue_ban', 'ademe_nom_commune_ban',
       'ademe_code_postal_ban', 'ademe_code_insee_ban',
       'ademe_surface_habitable_logement', 'ademe_etiquette_dpe',
       'ademe_etiquette_ges', 'ademe_conso_5_usages_par_m2_ep',
       'ademe_conso_5_usages_ep', 'ademe_conso_5_usages_par_m2_ef',
       'ademe_conso_5_usages_ef', 'ademe_type_batiment',
       'ademe_periode_construction', 'ademe_annee_construction',
       'ademe_date_etablissement_dpe', 'ademe_

In [ ]:
test = pd.read_csv(OUTPUT_PATH)
test

,enedis_address_ban,enedis_ban_lat,enedis_ban_lon,enedis_ban_id,enedis_numero_de_voie,enedis_type_de_voie,enedis_libelle_de_voie,enedis_code_commune,enedis_nom_commune,enedis_nombre_de_logements,...,ademe_date_visite_diagnostiqueur,ademe_date_derniere_modification_dpe,ademe_qualite_isolation_enveloppe,ademe_qualite_isolation_murs,ademe_qualite_isolation_menuiseries,ademe_type_installation_chauffage,ademe_type_installation_ecs,ademe_annee_ademe,annee_enedis,annee_ademe_matched
0,1 All G Anthonioz de Gaulle 93260 Les Lilas,48.879755,2.413890,93045_4700_00001,1.0,ALLEE,G ANTHONIOZ DE GAULLE,93045,Les Lilas,49,...,2021-09-16,2024-02-06,insuffisante,bonne,très bonne,individuel,individuel,2024,2019,2024
1,1 All Jean Baptiste Camille Corot 76120 Le Gra...,49.402653,1.037925,76322_0728_00001,1.0,ALLEE,JEAN BAPTISTE CAMILLE COROT,76322,Le Grand-Quevilly,14,...,2023-10-22,2024-02-06,moyenne,bonne,très bonne,collectif,collectif,2024,2019,2024
2,1 Allee Ambroise Pare 26200 Montélimar,44.559523,4.729666,26198_0123_00001,1.0,ALLEE,AMBROISE PARE,26198,MONTELIMAR,39,...,2022-01-09,2024-02-06,insuffisante,insuffisante,insuffisante,collectif,individuel,2024,2018,2024
3,1 Allee Andre Ampere 93110 Rosny-sous-Bois,48.882279,2.487403,93064_0060_00001,1.0,ALLEE,ANDRE AMPERE,93064,ROSNY-SOUS-BOIS,15,...,2022-06-07,2024-06-25,insuffisante,insuffisante,moyenne,collectif,collectif,2024,2018,2024
4,1 Allee Andre Crozillac 33610 Canéjan,44.750613,-0.643899,33090_3300_00001,1.0,ALLEE,ANDRE CROZILLAC,33090,CANEJAN,19,...,2021-12-29,2024-02-06,insuffisante,très bonne,très bonne,individuel,individuel,2024,2018,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85060,rue pierre vernier 25000 Besançon,47.240040,6.006692,25056_4050,14.0,RUE,PIERRE VERNIER,25056,BESANCON,15,...,2023-10-03,2024-02-06,insuffisante,très bonne,moyenne,individuel,collectif,2024,2018,2024
85061,rue pierre vernier 25000 Besançon,47.240040,6.006692,25056_4050,14.0,RUE,PIERRE VERNIER,25056,BESANCON,21,...,2023-10-03,2024-02-06,insuffisante,très bonne,moyenne,individuel,collectif,2024,2018,2024
85062,rue pierre vernier 25000 Besançon,47.240040,6.006692,25056_4050,14.0,RUE,PIERRE VERNIER,25056,BESANCON,15,...,2023-10-03,2024-02-06,insuffisante,très bonne,moyenne,individuel,collectif,2024,2018,2024
85063,rue romain roussel 25000 Besançon,47.256206,6.031216,25056_4548,37.0,RUE,ROMAIN ROUSSEL,25056,BESANCON,26,...,2023-07-26,2024-02-06,insuffisante,insuffisante,moyenne,individuel,individuel,2024,2018,2024


In [ ]:
# --------------------------
# CLEAN FINAL COLUMN NAMES
# --------------------------

# 1) RENAME ENEDIS (tu as dit OUI pour tout)
df_matched.rename(columns={
    "enedis_address_ban": "address_ban",
    "enedis_ban_lon": "longitude",
    "enedis_ban_lat": "latitude",
    "enedis_ban_id": "ban_id",
    "enedis_numero_de_voie": "numero_de_voie",
    "enedis_type_de_voie": "type_de_voie",
    "enedis_libelle_de_voie": "libelle_de_voie",
    "enedis_code_commune": "code_commune",
    "enedis_nom_commune": "nom_commune",
    "enedis_nombre_de_logements": "nombre_logements",
    "enedis_consommation_annuelle_totale_de_l_adresse_mwh": "conso_annuelle_mwh",
    "enedis_code_departement": "code_departement",
    "enedis_code_region": "code_region",
    "enedis_conso_par_logement_MWh": "conso_logement_mwh",
    "enedis_conso_par_logement_kWh": "conso_logement_kwh",
    "enedis_numero_de_voie_num": "numero_de_voie_num"
}, inplace=True)

# 2) SUPPRIMER colonnes ADEME que tu NE VEUX PAS garder
cols_to_drop = [
    "ademe_numero_voie_ban",
    "ademe_nom_rue_ban",
    "ademe_nom_commune_ban",
    "ademe_code_postal_ban"
]

df_matched.drop(columns=[c for c in cols_to_drop if c in df_matched.columns],
                inplace=True)

# 3) RENOMMER ADEME (on garde tout sauf ce que tu as supprimé)
df_matched.rename(columns={
    "ademe_numero_dpe": "numero_dpe",
    "ademe_code_insee_ban": "code_insee",
    "ademe_surface_habitable_logement": "surface_habitable",
    "ademe_etiquette_dpe": "etiquette_dpe",
    "ademe_etiquette_ges": "etiquette_ges",
    "ademe_conso_5_usages_par_m2_ep": "conso_5_usages_m2_ep",
    "ademe_conso_5_usages_ep": "conso_5_usages_ep",
    "ademe_conso_5_usages_par_m2_ef": "conso_5_usages_m2_ef",
    "ademe_conso_5_usages_ef": "conso_5_usages_ef",
    "ademe_type_batiment": "type_batiment",
    "ademe_periode_construction": "periode_construction",
    "ademe_annee_construction": "annee_construction",
    "ademe_date_etablissement_dpe": "date_etablissement_dpe",
    "ademe_date_fin_validite_dpe": "date_fin_validite_dpe",
    "ademe_date_reception_dpe": "date_reception_dpe",
    "ademe_date_visite_diagnostiqueur": "date_visite_diagnostiqueur",
    "ademe_date_derniere_modification_dpe": "date_modif_dpe",
    "ademe_annee_ademe": "annee_dpe",
    "annee_enedis": "annee_enedis",
    "annee_ademe_matched": "annee_dpe_matched"
}, inplace=True)

# 4) SAUVEGARDE
df_matched.to_csv(OUTPUT_PATH, index=False)

print("✔ Colonnes renommées et nettoyées")
print("📁 Fichier final sauvegardé dans :", OUTPUT_PATH)
print(df_matched.head())


✔ Colonnes renommées et nettoyées
📁 Fichier final sauvegardé dans : /content/drive/MyDrive/Hackathon_2025/datasets/clean/dbz.csv
                                         address_ban   latitude  longitude  \
0        1 All G Anthonioz de Gaulle 93260 Les Lilas  48.879755   2.413890   
1  1 All Jean Baptiste Camille Corot 76120 Le Gra...  49.402653   1.037925   
2             1 Allee Ambroise Pare 26200 Montélimar  44.559523   4.729666   
3         1 Allee Andre Ampere 93110 Rosny-sous-Bois  48.882279   2.487403   
4              1 Allee Andre Crozillac 33610 Canéjan  44.750613  -0.643899   

             ban_id  numero_de_voie type_de_voie              libelle_de_voie  \
0  93045_4700_00001             1.0        ALLEE        G ANTHONIOZ DE GAULLE   
1  76322_0728_00001             1.0        ALLEE  JEAN BAPTISTE CAMILLE COROT   
2  26198_0123_00001             1.0        ALLEE                AMBROISE PARE   
3  93064_0060_00001             1.0        ALLEE                 ANDRE AMPERE 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

OUTPUT_PATH ="/content/drive/MyDrive/Hackathon_2025/datasets/processed/dbz.csv"
test = pd.read_csv(OUTPUT_PATH)
test.columns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Index(['address_ban', 'latitude', 'longitude', 'ban_id', 'numero_de_voie',
       'type_de_voie', 'libelle_de_voie', 'code_commune', 'nom_commune',
       'nombre_logements', 'conso_annuelle_mwh', 'code_departement',
       'code_region', 'conso_logement_mwh', 'conso_logement_kwh',
       'numero_de_voie_num', 'numero_dpe', 'code_insee', 'surface_habitable',
       'etiquette_dpe', 'etiquette_ges', 'conso_5_usages_m2_ep',
       'conso_5_usages_ep', 'conso_5_usages_m2_ef', 'conso_5_usages_ef',
       'type_batiment', 'periode_construction', 'annee_construction',
       'date_etablissement_dpe', 'date_fin_validite_dpe', 'date_reception_dpe',
       'date_visite_diagnostiqueur', 'date_modif_dpe',
       'ademe_qualite_isolation_enveloppe', 'ademe_qualite_isolation_murs',
       'ademe_qualite_isolation_menuiseries',
       'ademe_type_installation_chauffage', 'ademe_type_installation_ecs',
       'annee_dpe', 'annee_enedis', 'annee_dpe_matched'],
      dtype='object')

In [ ]:
test.to_csv(OUTPUT_PATH, index=False)

In [12]:
test[test["address_ban"] == "89 Rue du Marais 59160 Lille"]

,address_ban,latitude,longitude,ban_id,numero_de_voie,type_de_voie,libelle_de_voie,code_commune,nom_commune,nombre_logements,...,date_visite_diagnostiqueur,date_modif_dpe,ademe_qualite_isolation_enveloppe,ademe_qualite_isolation_murs,ademe_qualite_isolation_menuiseries,ademe_type_installation_chauffage,ademe_type_installation_ecs,annee_dpe,annee_enedis,annee_dpe_matched
76986,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,15,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76987,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,19,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76988,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,21,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76989,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,21,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76990,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,21,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76991,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,22,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76992,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,22,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76993,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,21,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76994,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,42,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024
76995,89 Rue du Marais 59160 Lille,50.636565,3.020981,59350_9532_00089,89.0,RUE,DU MARAIS,59350,LILLE,33,...,2023-08-30,2024-02-06,insuffisante,insuffisante,moyenne,collectif,individuel,2024,2018,2024


In [11]:
counts_df = test['address_ban'].value_counts().reset_index()
counts_df.columns = ['address_ban', 'nb_occurrences']
counts_df.head()

,address_ban,nb_occurrences
0,89 Rue du Marais 59160 Lille,11
1,46 Rue du Mont Bart 25200 Montbéliard,10
2,16 Rue Maria Chapdelaine 29000 Quimper,8
3,11 Rue Gustave Masson 10000 Troyes,8
4,90 Avenue Chomedey de Maisonneuve 10000 Troyes,8


In [18]:
ademe = pd.read_csv("/content/drive/MyDrive/Hackathon_2025/datasets/clean/ademe_final.csv")
ademe["date_derniere_modification_dpe"].unique()

array(['2024-06-25', '2024-02-06', '2023-10-26', '2024-09-17',
       '2025-01-20', '2024-06-26', '2024-01-23', '2023-11-08',
       '2023-11-17', '2023-11-12', '2023-12-05', '2023-12-22',
       '2023-11-13', '2023-11-15', '2023-12-11', '2023-11-20',
       '2023-10-28', '2025-01-15', '2023-12-13', '2023-12-08',
       '2023-10-27', '2023-12-15'], dtype=object)